# <center> <b> Mood Based Content Recommendation Using Python </center> </b>

In [2]:
!pip install gensim

     ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/24.0 MB ? eta -:--:--
     --------------------------------------- 0.0/24.0 MB 325.1 kB/s eta 0:01:14
     --------------------------------------- 0.0/24.0 MB 325.1 kB/s eta 0:01:14
     --------------------------------------- 0.0/24.0 MB 325.1 kB/s eta 0:01:14
     --------------------------------------- 0.0/24.0 MB 325.1 kB/s eta 0:01:14
     --------------------------------------- 0.0/24.0 MB 325.1 kB/s eta 0:01:14
     --------------------------------------- 0.0/24.0 MB 325.1 kB/s eta 0:01:14
     --------------------------------------- 0.1/24.0 MB 155.6 kB/s eta 0:02:34
     --------------------------------------- 0.1/24.0 MB 208.4 kB/s eta 0:01:55
     --------------------------------------- 0.1/24.0 MB 218.5 kB/s eta 0:01:50
     --------------------------------------- 0.1/24.0 MB 218.5 kB/s eta 0:01:50
     --------------------------------------- 0.1/24.0 MB


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import gensim.downloader as api

# Download and save the word vectors locally
word_vectors = api.load("glove-wiki-gigaword-100")
word_vectors.save("glove-wiki-gigaword-100.kv")

[==================================================] 100.0% 128.1/128.1MB downloaded


In [2]:
import gensim.downloader as api
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cosine

# Load the dataset
dataset_path = "Mood_Recommendations_Dataset.csv"
data = pd.read_csv(dataset_path)

# Load pre-trained word vectors (GloVe or Word2Vec)
# word_vectors = api.load("glove-wiki-gigaword-100")  # Example: GloVe vectors

# Load pre-installed vectors from a local file
word_vectors = KeyedVectors.load("glove-wiki-gigaword-100.kv", mmap='r')

# Function to compute soft cosine similarity
def soft_cosine_similarity(text1, text2, word_vectors):
    # Tokenize the text (split into words and remove stopwords)
    text1_tokens = text1.lower().split()
    text2_tokens = text2.lower().split()

    # Filter tokens that are in the word vectors' vocabulary
    valid_tokens1 = [token for token in text1_tokens if token in word_vectors]
    valid_tokens2 = [token for token in text2_tokens if token in word_vectors]

    if not valid_tokens1 or not valid_tokens2:
        return 0.0  # Return 0 if no valid tokens for comparison

    # Create the vector representations for each text based on word vectors
    vector1 = sum(word_vectors[token] for token in valid_tokens1) / len(valid_tokens1)
    vector2 = sum(word_vectors[token] for token in valid_tokens2) / len(valid_tokens2)

    # Compute cosine similarity between the two vectors
    similarity = 1 - cosine(vector1, vector2)
    return similarity

# Function to recommend activities using soft cosine similarity
def recommend_activities(user_message, mood, top_n=5):
    # Filter dataset by mood
    mood_data = data[data['Mood'].str.lower() == mood.lower()]
    # Combine Recommendation and Details for text matching
    mood_data['Combined_Text'] = mood_data['Recommendation'] + " " + mood_data['Details']

    # Calculate similarity scores using soft cosine similarity
    similarities = mood_data['Combined_Text'].apply(lambda x: soft_cosine_similarity(user_message, x, word_vectors))

    # Add similarity scores
    mood_data = mood_data.copy()
    mood_data['Similarity'] = similarities

    # Get top N unique recommendations based on similarity
    top_recommendations = (
        mood_data[['Mood', 'Category', 'Recommendation', 'Details', 'Similarity']]
        .drop_duplicates(subset=['Recommendation'])
        .nlargest(top_n, 'Similarity')
    )

    return top_recommendations[['Mood', 'Category', 'Recommendation', 'Details','Similarity']]

# Example usage
user_message = "I feeling down today."
mood = "Sad"
top_activities = recommend_activities(user_message, mood)
top_activities

C:\Users\Jayachandru\AppData\Local\Temp\ipykernel_6704\1203087711.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mood_data['Combined_Text'] = mood_data['Recommendation'] + " " + mood_data['Details']


,Mood,Category,Recommendation,Details,Similarity
59,Sad,Content,Look at old happy photos,Remind yourself of good memories.,0.869916
50,Sad,Activity,Go for a walk,Walking can help clear your mind and lift your...,0.859786
56,Sad,Activity,Practice gratitude journaling,Write down things you're grateful for.,0.853989
53,Sad,Activity,Call a close friend,Share your feelings with someone who understands.,0.845091
52,Sad,Music,Listen to uplifting music,Play songs that remind you of happy moments.,0.834228


In [5]:
top_activities.to_json(orient='records')

'[{"Mood":"Sad","Category":"Content","Recommendation":"Look at old happy photos","Details":"Remind yourself of good memories.","Similarity":0.8699159622},{"Mood":"Sad","Category":"Activity","Recommendation":"Go for a walk","Details":"Walking can help clear your mind and lift your mood.","Similarity":0.8597858548},{"Mood":"Sad","Category":"Activity","Recommendation":"Practice gratitude journaling","Details":"Write down things you\'re grateful for.","Similarity":0.8539891839},{"Mood":"Sad","Category":"Activity","Recommendation":"Call a close friend","Details":"Share your feelings with someone who understands.","Similarity":0.8450909853},{"Mood":"Sad","Category":"Music","Recommendation":"Listen to uplifting music","Details":"Play songs that remind you of happy moments.","Similarity":0.8342277408}]'